In [1]:
from pathlib import Path
import sys
sys.path.append('..')

from tools import get_input, profile

test_raw = [[field for field in line ] for line in (Path().parent / "test.txt").read_text(encoding="utf-8").splitlines()]
input_raw = [[field for field in line ] for line in get_input(6).splitlines()]

In [2]:
def find_start(data: list[list[str]]) -> tuple[str, int, int]:
    for i, line in enumerate(data):
        for j, field in enumerate(line):
            if field in "^v<>":
                return field, i, j

def solution_1(data: list[list[str]], start: tuple[str, int, int]) -> int:
    dirs = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}
    symbol = ["^", ">", "v", "<"]
    field, i, j = start
    mapped = [line.copy() for line in data]
    dir = dirs[field]
    res = 1
    while 0 <= i + dir[0] < len(mapped) and 0 <= j + dir[1] < len(mapped[0]):
        if mapped[i + dir[0]][j + dir[1]] == "#":
            field = symbol[(symbol.index(field) + 1) % 4]
            dir = dirs[field]
            mapped[i][j] += field
            continue
        i, j = i + dir[0], j + dir[1]
        if mapped[i][j] == ".":
            mapped[i][j] = field
            res += 1
        elif field in mapped[i][j]:
            return -1, mapped
        else:
            mapped[i][j] += field
    return res, mapped

res, mapped = solution_1(test_raw, find_start(test_raw))
assert res == 41
res, big_map = solution_1(input_raw, find_start(input_raw))
res # 4890

4890

In [3]:
mapped

[['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '^>', '>', '>', '>', '>v', '#'],
 ['.', '.', '.', '.', '^', '.', '.', '.', 'v', '.'],
 ['.', '.', '#', '.', '^', '.', '.', '.', 'v', '.'],
 ['.', '.', '^>', '>', '^>', '>', '>v', '#', 'v', '.'],
 ['.', '.', '^', '.', '^', '.', 'v', '.', 'v', '.'],
 ['.', '#', '<^', '<', '^<', '<', '<v', '<', 'v<', '.'],
 ['.', '^>', '>', '>', '>', '>', 'v>', '>v', '#', '.'],
 ['#', '<^', '<', '<', '<', '<', 'v<', 'v', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '#', 'v', '.', '.']]

In [4]:
@profile
def solution_2(data: list[list[str]]):
    cnt = 0
    start = find_start(data)
    for i in range(len(data)):
        for j in range(len(data[0])):
            if data[i][j] == ".":
                data[i][j] = "#"
                if solution_1(data, start)[0] == -1:
                    cnt += 1
                data[i][j] = "."
    return cnt
assert solution_2(test_raw) == 6
solution_2(input_raw)  # 1995

0.0007860660552978516
12.101609945297241


1995

In [5]:
@profile
def solution_2_optimized(data: list[list[str]], mapped: list[list[str]]):
    res = 0
    test = set()
    start = find_start(data)
    test = {
        (i, j)
        for i in range(len(mapped))
        for j in range(len(mapped[0]))
        if mapped[i][j] not in ["#", "."]
    }
    test.remove((start[1], start[2]))
    for x, y in test:
        data[x][y] = "#"
        if solution_1(data, start)[0] == -1:
            res += 1
        data[x][y] = "."
    return res


assert solution_2_optimized(test_raw, mapped) == 6
assert solution_2_optimized(input_raw, big_map) == 1995

0.00032591819763183594
2.6343352794647217
